In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import json
import time
from IPython.display import display,HTML

from tqdm import tqdm
import pandas as pd
import requests

In [ ]:
%%appyter hide_code
{% do SectionField(
    name='drug_list_section',
    title='1. Submit Your Drug List',
    subtitle='Submit a list of small molecules for enrichment analysis using the Drugmonizome data API.',
    img = 'load_icon.png'
)%}

{% do SectionField(
    name='library_section', 
    title = '2. Choose Drug Set Libraries', 
    subtitle = 'Select Drugmonizome drug set libraries.', 
    img = 'drugmonizome_logo.png'
) %}

{% do SectionField(
    name='background_section', 
    title = '(Optional) Submit a Background Set', 
    subtitle = 'Upload a text file containing a list of background items to use as a filter for both the input\
                and the library sets. You can also copy and paste your background items into the text box below\
                (one item per row). The default is no background and no filtering out any items.', 
    img = 'settings.png'
) %}

In [ ]:
%%appyter hide_code
{% set drug_list = TabField(
    name='drug_list_field',
    label='Drug List',
    default='Paste',
    description='Paste or upload your drug list',
    choices={
        'Paste': [
            TextListField(
                name='drug_list_input',
                label='Drug List',
                description='Paste your drug list (One drug per row).',
                default=['lopinavir','remdesivir','mefloquine','loratadine','almitrine','camostat',
                         'pevonedistat','octenidine','nafamostat','papaverine','apixaban','alvocidib',
                         'methylene blue','nemetine','cycloheximide','amuvatinib','thimerosal',
                         'homoharringtonine','posaconazole','polidocanol','drotaverine','ketoconazole',
                         'alisporivir','regorafenib','loteprednol','lidoflazine','chlormidazole','tyrphostin',
                         'avasimibe','brexpiprazole','sorafenib','lonafarnib','dapivirine','flunarizine',
                         'jte-013','ravuconazole','mibampator','oxiconazole','zk-93423','cetylpyridinium',
                         'avatrombopag','ph-797804','etifoxine','dcpib','cc-223','vatalanib','tioguanine',
                         'amg-9810','pexidartinib','carboxyamidotriazole','ac1ndss5','adoprazine','ai-10-49',
                         'bp-897','gsk2606414','ipag','ide225','lgk-974','ly2228820','nsc319726','pf-670462',
                         'sb-612111','vlx600','cbipes','cloconazole','ethaverine','hematoporphyrin',
                         'thioguanosine','harringtonine'  
                ],
            ),
        ],
        
        'Upload':[
            FileField(
                name='drug_list_filename',
                label='Drug List File',
                default='',
                description='Upload your drug list as a text file (One drug per row).',
            ),
        ],
    },
    section = 'drug_list_section',
)%}

{% set drug_identifier = ChoiceField(
    name='drug_identifier',
    label='Drug Identifier Type',
    default= 'Name',
    description='Choose the appropriate identifier type for the drug list',
    choices=[
        'Name',
        'InChI_key',
        'SMILES'
    ],
    section='drug_list_section'

)%}

{% set drug_set_libraries = MultiCheckboxField(
    name='drug_set_libraries',
    label='Drug Set Library',
    default= ['SIDER_side_effects_drugsetlibrary',
              'L1000FWD_GO_Biological_Processes_drugsetlibrary_down',
              'L1000FWD_GO_Biological_Processes_drugsetlibrary_up'],
    description='Choose the drug set library to use as background for enrichment analysis',
    choices=['KinomeScan_kinase_drugsetlibrary',
             'L1000FWD_GO_Cellular_Component_drugsetlibrary_up',
             'L1000FWD_KEGG_Pathways_drugsetlibrary_down',
             'L1000FWD_signature_drugsetlibrary_up',
             'Geneshot_associated_drugsetlibrary',
             'Geneshot_predicted_generif_drugsetlibrary',
             'SIDER_indications_drugsetlibrary',
             'L1000FWD_GO_Molecular_Function_drugsetlibrary_up',
             'L1000FWD_GO_Molecular_Function_drugsetlibrary_down',
             'L1000FWD_KEGG_Pathways_drugsetlibrary_up',
             'SIDER_side_effects_drugsetlibrary',
             'DrugRepurposingHub_target_drugsetlibrary',
             'L1000FWD_GO_Biological_Processes_drugsetlibrary_down',
             'L1000FWD_GO_Biological_Processes_drugsetlibrary_up',
             'L1000FWD_GO_Cellular_Component_drugsetlibrary_down',
             'ATC_drugsetlibrary',
             'Drugbank_smallmolecule_target_drugsetlibrary',
             'STITCH_target_drugsetlibrary',
             'Geneshot_predicted_autorif_drugsetlibrary',
             'Drugbank_smallmolecule_enzyme_drugsetlibrary',
             'PharmGKB_OFFSIDES_side_effects_drugsetlibrary',
             'CREEDS_signature_drugsetlibrary_down',
             'Geneshot_predicted_tagger_drugsetlibrary',
             'RDKIT_maccs_fingerprints_drugsetlibrary',
             'CREEDS_signature_drugsetlibrary_up',
             'DrugCentral_target_drugsetlibrary',
             'L1000FWD_signature_drugsetlibrary_down',
             'L1000FWD_predicted_side_effects',
             'Drugbank_smallmolecule_carrier_drugsetlibrary',
             'PubChem_fingerprints_drugsetlibrary',
             'Geneshot_predicted_enrichr_drugsetlibrary',
             'DrugRepurposingHub_moa_drugsetlibrary',
             'Geneshot_predicted_coexpression_drugsetlibrary',
             'Drugbank_smallmolecule_transporter_drugsetlibrary',
             'PharmGKB_snp_drugsetlibrary'
        
    ],
    section='library_section'

)%}

{% set background = TabField(
    name='background_kind',
    label='Background',
    default='Paste',
    description='Paste or upload your background set',
    choices={
        'Paste': [
            TextListField(
                name='background_input',
                label='Background Set',
                default=[],
                description='Paste your background set (one item per row). Names in the background set should match the names in the drug set library.'
            ),
        ],
        'Upload': [
            FileField(
                name='background_filename',
                label='Background File',
                default='',
                description='Upload your background set as a text file (one item per row). Names in the background set should match the names in the drug set library.'
            ),
        ],
    },
    section = 'background_section',
) %}

### Import drug list

In [ ]:
%%appyter code_exec
{% if drug_list.raw_value == 'Paste' %}
drug_list = {{drug_list.value[0]}}
{% else %}
with open({{drug_list.value[0]}}, 'r') as f:
    drug_list = list(set(map(str.strip, f)))
{% endif %}
{% if drug_identifier.value == 'Name' %}
drug_list = [x.lower() for x in drug_list]
{% endif %}

In [ ]:
%%appyter code_exec
# Retrieve entity UUIDs
url = 'https://maayanlab.cloud/drugmonizome/metadata-api/entities/find'
filter_body = {
    "filter": {
        "where": {
            "meta.{{drug_identifier.value}}": {
                "inq": drug_list
            }
        }
    }
}
entities = requests.post(url, json=filter_body)

# Create lookup of UUIDs to chosen identifier
entity_lookup = {item['id']:item['meta']['{{drug_identifier.value}}'] for item in entities.json()}

In [ ]:
%%appyter markdown
{% if background.value[0].raw_value %}
### Import background set
{% endif %}

In [ ]:
%%appyter code_exec
{% if background.value[0].raw_value %}
{% if background.raw_value == 'Paste' %}
background_set = {{background.value[0]}}
{% else %}
with open({{background.value[0]}}, 'r') as f:
    background_set = list(set(map(str.strip, f)))
{% endif %}
{% if drug_identifier.value == 'Name' %}
background_set = [x.lower() for x in background_set]
{% endif %}
{% endif %}

In [ ]:
%%appyter code_exec
{% if background.value[0].raw_value %}
# Retrieve background set UUIDs
url = 'https://maayanlab.cloud/drugmonizome/metadata-api/entities/find'
filter_body = {
    "filter": {
        "where": {
            "meta.{{drug_identifier.value}}": {
                "inq": background_set
            }
        }
    }
}
background_entities = requests.post(url, json=filter_body).json()
background_entities = [i['id'] for i in background_entities]
{% endif %}

### Query Drugmonizome data API and create enrichment table(s)

In [ ]:
%%appyter code_exec
# send entity UUIDs for enrichment
enrichment_endpoint = 'https://maayanlab.cloud/drugmonizome/data-api/api/v1/enrich/overlap'
signatures_endpoint = 'https://maayanlab.cloud/drugmonizome/metadata-api/signatures/%s'

for library in {{drug_set_libraries}}:
    output = []
    payload = {'database': library,
               'entities': list(entity_lookup.keys()),
                {% if background.value[0].raw_value %}
                'background': background_entities
                {% endif %}}
    result = requests.post(enrichment_endpoint, json = payload)

    for item in tqdm(result.json()['results']):
        if item['p-value'] < 0.05:
            signature = requests.get(signatures_endpoint % item['uuid']).json()
            # Create output object with UUIDs for entities and signatures decoded
            output.append({'term': signature['meta']['Term'][0]['Name'],
                           'p-value': item['p-value'],
                           'p-value-bonferroni':item['p-value-bonferroni'],
                           'false discovery rate':item['fdr'],
                           'drug set size': item['setsize'],
                           'overlapping_drugs': [entity_lookup.get(x,x) for x in item['overlap']]
                              })
    enrichment_table = pd.DataFrame(output).head(20)
    enrichment_table.to_csv(f'enrichment_table_{library}.tsv', sep = '\t', index = False)
    display(HTML(f"<div style='font-size:1.5rem; padding:1rem 0;'>Top 20 Enriched Terms from {library}</div><a target=_blank href=./enrichment_table_{library}.tsv>enrichment_table_{library}.tsv</a>"))
    display(HTML(enrichment_table.to_html(index=False)))